In [21]:
import os
import re
import csv
import time
import difflib

directory = r'C:\Users\Neuromancer\Desktop\Presentation'

# scan through directory finding all unprocessed files
def find_files_to_edit(directory):
    # filename ends with this datetime formatting
    name_finder = re.compile(r'_\d\d\d\d-\d\d-\d\d-\d\d\d\d\d\d$')
    file_list = []
    for filename in os.listdir(directory):
        if name_finder.search(filename) is not None:
            file_list.append(directory + '\\' + filename)
            print('Added file')
        else:
            print('Not adding file: ' + filename)

    return file_list

unprocessed_files = find_files_to_edit(directory)

Not adding file: .ipynb_checkpoints
Not adding file: AHE.ipynb
Not adding file: AMR-USMR.ipynb
Not adding file: AnalysisFunctions.py
Not adding file: Data_Cleaner.ipynb
Not adding file: Messy_AHE
Not adding file: Pandas_Tutorial.ipynb
Not adding file: Pulse_Switching.ipynb
Added file
Not adding file: __pycache__


In [39]:
# get headers, check if x2 data is present
new_xy_headers = []
old_headers = []
sep_list = []
sep_index_list = []
with open(unprocessed_files[0], 'r') as f:
    head = [next(f) for x in range(7)]
    if len(head[6].split(' ')) == 4:
        x2_flag = True
    else:
        x2_flag = False
    # check headers
    for i in range(6):
        words = head[i].split(' ')
        if words[0] == 'Number':  # last line, get new headers for x and y columns
            print(head[i])
            header_index = i # index of last header value
            new_xy_headers.append(input('Input X Values Title: '))
            new_xy_headers.append(input('Input Y Values Title: '))
            if x2_flag:  # check if x2 title is needed
                new_xy_headers.append(input('Input X2 Values Title: '))
            break
        elif words[0] != '\n':
            print(head[i])
            q = input('Do you wish to seperate these headers? (y/n): ')
            if q == 'y' or q == 'Y':
                # i.e. I am Neil, he is bob. sep = 'he' results in "I am Neil, " and 'he is bob.'
                sep = input('Type the word to seperate from: ')  # character to seperate two headers
                sep_list.append(sep) 
                sep_index_list.append(i)
                old_headers.append(head[i].split(sep)[0])
                old_headers.append(sep + head[i].split(sep)[1].strip('\n'))
            else:
                old_headers.append(head[i].strip('\n'))
        else:
            pass
    f.close()
                
print(old_headers)

Hx field: 0.0(Oe)

Do you wish to seperate these headers? (y/n): n
Applied current: 2.0(mA)

Do you wish to seperate these headers? (y/n): n
Initial Resistance: 2715.0136Ohm

Do you wish to seperate these headers? (y/n): n
Number Hx Field(Oe) Resistance(Ohm) Real Applied Field (Oe)

Input X Values Title: X
Input Y Values Title: Y
Input X2 Values Title: X2
['Hx field: 0.0(Oe)', 'Applied current: 2.0(mA)', 'Initial Resistance: 2715.0136Ohm']


In [41]:
new_headers = []
delete_strings = []
for string in old_headers:
    print(string)
    new_headers.append(input('Input new column header: '))
    delete_strings.append(input('Type string elements to remove: '))
    
for ind, string in enumerate(delete_strings):
    print(f'For new header of {new_headers[ind]} deleteing string: {string}')

Hx field: 0.0(Oe)
Input new column header: Hz
Type string elements to remove: Hx field: (Oe)
Applied current: 2.0(mA)
Input new column header: Current
Type string elements to remove: Applied current: (mA)
Initial Resistance: 2715.0136Ohm
Input new column header: Init R
Type string elements to remove: Initial Resistance: Ohm
Deleteing string: Hx field: (Oe) for new header of Hz
Deleteing string: Applied current: (mA) for new header of Current
Deleteing string: Initial Resistance: Ohm for new header of Init R


In [57]:
# find the elements of s1 that are not in s2
def remove_strings(s1, s2):
    val = ''
    for character in difflib.ndiff(s1, s2):
        if character[0] == '+' and character[-1] != ' ':
            val += character[-1]
    return val

for file in unprocessed_files:
    misc_val = []
    x = []
    y = []
    x2 = []
    # extract header values and xy data
    with open(file, 'r') as f:
        data = f.readlines()
        delete_str_ind = 0
        for i in range(header_index): # process headers
            if data[i] == '\n':
                pass
            elif i in sep_list:  # seperate and then extract values
                s1 = data[i].split(sep_index_list[sep_list.index(i)])[0]
                s2 = sep_index_list[sep_list.index(i)] + data[i].split(sep_index_list[sep_list.index(i)])[1]
                misc_val.append(remove_strings(delete_strings[delete_str_ind], s1))
                print(f'Removing {delete_strings[delete_str_ind]} from {s1}')
                delete_str_ind += 1
                misc_val.append(remove_strings(delete_strings[delete_str_ind], s2))
                print(f'Removing {delete_strings[delete_str_ind]} from {s2}')
                delete_str_ind += 1
            else:  # extract values
                misc_val.append(remove_strings(delete_strings[delete_str_ind], data[i].strip('\n')))
                print(f'Removing {delete_strings[delete_str_ind]} from {data[i]}')
                delete_str_ind += 1
        
        for ind, elem in enumerate(misc_val):
            print(f'{new_headers[ind]} with value {elem}')
            
        for n in range(header_index+1, len(data)): # extract xy data
            d = data[n].split(' ')
            x.append(d[1])
            y.append(d[2])
            if x2_flag:
                x2.append(d[3])
        f.close()
    
    # write data to csv
    with open(file+'.csv', 'w', newline='', encoding='utf-8') as csvfile:
        rewriter = csv.writer(csvfile, dialect='excel')
        rewriter.writerow(new_xy_headers + new_headers)
        for counter, elem in enumerate(x):
            if x2_flag:
                rewriter.writerow([elem, y[counter], x2[counter]] + misc_val)
            else:
                rewriter.writerow([elem, y[counter]] + misc_val)
        csvfile.close()
    print(f'Saved file {file}.csv')

Removing Hx field: (Oe) from Hx field: 0.0(Oe)

Removing Applied current: (mA) from Applied current: 2.0(mA)

Removing Initial Resistance: Ohm from Initial Resistance: 2715.0136Ohm

Hz with value 0.0
Current with value 2.0
Init R with value 2715.0136
Saved file C:\Users\Neuromancer\Desktop\Presentation\Sample Name_AMR_Hx_scan_0.0Oe_2.0mA_2019-11-20-155421.csv


3
